In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# File path
file_dir = '.\\data\\source\\6_dB_pump\\pump\\id_00\\normal\\'
file_name = '00000156.wav'
file_path = os.path.join(file_dir, file_name)

In [ ]:
# Load the audio file 
signal, fs = librosa.load(file_path, sr=None, mono=False)
signal = signal[0, :]
print(f"File duration: {librosa.get_duration(filename=file_path)} s, fs={fs} Hz")

In [ ]:
# Plot the signal
plt.figure(figsize=(16,5))
plt.subplot(111)
librosa.display.waveplot(signal, sr=fs)
plt.title(f"Audio file {file_path} with fs={fs} Hz")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
np.mean(signal)
np.std(signal)


In [ ]:
# Plot multiple signals
signal_multi_ch, sr = librosa.load(file_path, sr=None, mono=False)
fig = make_subplots(rows=4, cols=2)
for no_sig in range(signal_multi_ch.shape[0]):
    fig.add_trace(
    go.Scatter(y=signal_multi_ch[no_sig, :]),
    row=no_sig%4+1, col=(no_sig)//4+1)
fig.show()

In [ ]:
# Compute and plot the FFT
sig_fft = np.fft.fft(signal, n=1024)
freq = np.linspace(0, fs, len(sig_fft))

sig_fft_left = sig_fft[:len(freq)//2]
freq_left = freq[:len(freq)//2]

fig = px.line(x=freq_left, y=abs(sig_fft_left))
fig.update_layout(title="FFT of the signal",
        yaxis_title="abs(fft)",
        xaxis_title="Frequency (Hz)")

In [ ]:
# Compute and plot the FFT for multiple signals
fig = go.Figure()
for no_sig in range(signal_multi_ch.shape[0]):
    
    sig_fft = np.fft.fft(signal_multi_ch[no_sig, :], n=256)
    freq = np.linspace(0, fs, len(sig_fft))

    sig_fft_left = sig_fft[:len(freq)//2]
    freq_left = freq[:len(freq)//2]
    
    fig.add_trace(
    go.Scatter(x=freq_left, y=abs(sig_fft_left))
    )

fig.update_layout(title="FFT of all channels",
                  yaxis_title="abs(fft)",
                  xaxis_title="Frequency (Hz)")


fig.show()

In [ ]:
help(librosa.filters.mel)

In [ ]:
# Mel filter banks
n_fft = 1024
filter_banks = librosa.filters.mel(n_fft=n_fft, sr=fs, n_mels=10)
print(filter_banks.shape)

plt.figure(figsize=(10,4))
librosa.display.specshow(filter_banks, sr=fs, x_axis='linear')
plt.colorbar(format="%.2e")
plt.show()

In [ ]:
def get_melspectogram(file_path, window, no_channel=None, overlap=None, n_fft=None, n_mels=32, machine=None):
    
    # Load the signal
    signal, fs = librosa.load(file_path, sr=None, mono=False)
    
    # Default channel no selected based on machine
    channel_dict = {'pump': 3, 'valve': 1, 'slider': 7, 'fan': 5}
    if no_channel == None:
        if machine == None:
            no_channel = 1
        else:
            no_channel = channel_dict[machine]-1
        signal = signal[no_channel, :]
    else:
        signal = signal.mean(axis=0)
    
    # Window length in samples
    window_length = int(window*fs)
    
    # Default overlap is 50% of the window size
    if overlap==None:
        overlap = 0.5
    
    # Overlap in samples
    overlap_length = int(window_length*overlap)
    
    # Hop length in samples
    hop_length = window_length-overlap_length
    
    # Default n_fft is the smallest power of 2 larger than win_length
    if n_fft==None:
        n_fft = int(2**np.ceil(np.log2(window_length)))
   
    # Compute mel spectogram
    mel_spect = librosa.feature.melspectrogram(y=signal, sr=fs, 
                                               win_length=window_length, 
                                               hop_length=hop_length,
                                               n_fft=n_fft,
                                               n_mels=n_mels)
    # Mel spectogram in decibels
    mel_spect_db = librosa.power_to_db(mel_spect, ref=1.0, amin=sys.float_info.epsilon, top_db=np.inf)

    params = {}
    params['window'] = window
    params['window_length'] = window_length
    params['overlap'] = overlap
    params['overlap_length'] = overlap_length
    params['hop_length'] = hop_length
    params['n_fft'] = n_fft
    
    return mel_spect, mel_spect_db, params

# Compute and plot mel spectogram
window = 1024/fs # 2
n_mels = 64
mel_spect, mel_spect_db, params = get_melspectogram(file_path, window=window, overlap=0.5, n_mels=n_mels) 

print(params)

print(f"Size of mel_spect: {mel_spect.shape}, total number of points: {mel_spect.size}")
librosa.display.specshow(mel_spect_db, sr=fs, hop_length=params['overlap_length'], x_axis='time', y_axis='mel');
# times = librosa.times_like(mel_spect, sr=fs, hop_length=params['overlap_length'])
# librosa.display.specshow(mel_spect_db, sr=fs, hop_length=params['overlap_length'], x_coords = times, y_axis='mel');

plt.title(f"Mel Spectrogram, window size {params['window']} s");
plt.colorbar(format='%+2.0f dB');

In [ ]:
n_frames = 5
dims = n_mels * n_frames
vectorarray_size = len(mel_spect_db[0, :]) - n_frames + 1

print("vectorarray_size:", vectorarray_size)

vectorarray = np.zeros((vectorarray_size, dims), float)
print("vectorarray: ", vectorarray.shape)

for t in range(n_frames):
    print(t, t + vectorarray_size)
    print(n_mels*t, n_mels*(t + 1))
    print("\n")
    vectorarray[:, n_mels*t : n_mels*(t + 1)] = mel_spect_db[:, t: t + vectorarray_size].T


vectorarray.shape

In [ ]:
librosa.display.specshow(vectorarray)

In [ ]:
win_length = params['window_length']
n_fft = params['n_fft']
hop_length = params['hop_length']
n_mfcc = 16
mfcc = librosa.feature.mfcc(signal, n_fft=n_fft, win_length=win_length, hop_length=hop_length)

librosa.display.specshow(np.log10(abs(mfcc)), sr=fs, hop_length=hop_length)
plt.title("Spectogram of the signal")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.show()

In [ ]:
def extract_features(file_path):    
    audio, sample_rate = librosa.load(file_path) 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, win_length=win_length, hop_length=hop_length, n_mfcc=64)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs, mfccs_processed

mfccs, mfccs_processed = extract_features(file_path)

mfccs.shape

In [ ]:
mfccs_processed.shape